In [1]:
import matplotlib.pyplot as plt
import mpmath as mp
mp.mp.dps = 30 #higher precision
import pandas as pd
from PyMatching import *
import scipy

In [2]:
boundary_type = "periodic"
batch_results = []
probabilities = [0.1, 0.11, 0.12, 0.13]
sizes = [4, 6]
intervals = 10
iterations = 1000
overlap = 0.25
walkers = 8
alpha = 0.8
beta = 1e-8
exchange = 20

In [3]:
low_temp_dict = {}
nish_temp_dict = {}

for size in sizes:

    low_temp_dict[size] = {}
    nish_temp_dict[size] = {}

    for p in probabilities:
        filename = f"../results_curves/periodic/X_{size}_Y_{size}/prob_{p:.6f}/free_energies_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset_{exchange}.pickle"

        df = pd.read_pickle(filename)

        nish_curve = mp.mpf(1)
        low_temp_curve = mp.mpf(1)

        # Group by 'histogram_seed' to avoid explicit loops
        grouped_df = df.groupby('histogram_seed')

        for _, df_run in grouped_df:
            # Vectorized comparison for 'free_energy_nish_temp'
            free_energy_I_nish = df_run.loc[df_run['error'] == 'I', 'free_energy_nish_temp'].values[0]
            other_errors_nish = df_run.loc[df_run['error'] != 'I', 'free_energy_nish_temp']

            nish_curve -= mp.mpf(int((free_energy_I_nish < other_errors_nish).any())) / mp.mpf(len(grouped_df))

            # Vectorized comparison for 'free_energy_low_temp'
            free_energy_I_low = df_run.loc[df_run['error'] == 'I', 'free_energy_low_temp'].values[0]
            other_errors_low = df_run.loc[df_run['error'] != 'I', 'free_energy_low_temp']

            low_temp_curve -= mp.mpf(int((free_energy_I_low < other_errors_low).any())) / mp.mpf(len(grouped_df))

        number_success_nish = round(len(grouped_df) * nish_curve)
        number_success_low_temp = round(len(grouped_df) * low_temp_curve)

        number_failure_nish = len(grouped_df) - number_success_nish
        number_failure_low_temp = len(grouped_df) - number_success_low_temp

        lower_bound_nish = nish_curve - scipy.stats.beta.ppf(0.025, 0.5 + number_success_nish, 0.5 + number_failure_nish)
        lower_bound_low_temp = low_temp_curve - scipy.stats.beta.ppf(0.025, 0.5 + number_success_low_temp, 0.5 + number_failure_low_temp)

        upper_bound_nish = scipy.stats.beta.ppf(0.975, 0.5 + number_success_nish, 0.5 + number_failure_nish) - nish_curve
        upper_bound_low_temp = scipy.stats.beta.ppf(0.975, 0.5 + number_success_low_temp, 0.5 + number_failure_low_temp) - low_temp_curve

        low_temp_dict[size][p] = [low_temp_curve, lower_bound_low_temp, upper_bound_low_temp]
        nish_temp_dict[size][p] = [nish_curve, lower_bound_nish, upper_bound_nish]

In [4]:
low_temp_dict

{4: {0.1: [mpf('0.741391627375339334190598657022038'),
   mpf('0.0103566104487493253117374304731317'),
   mpf('0.0101570121630441330495248795903347')],
  0.11: [mpf('0.698571428571428571428571428541943'),
   mpf('0.0108301211393237511210989915620645'),
   mpf('0.0106659523122795636090082943060792')],
  0.12: [mpf('0.656571428571428571428571428537835'),
   mpf('0.0111868424782492914865932626579892'),
   mpf('0.0110573968563601032449079347262155')],
  0.13: [mpf('0.615428571428571428571428571390953'),
   mpf('0.0114424335352861709884970358105248'),
   mpf('0.0113470026868116653823952739769255')]},
 6: {0.1: [mpf('0.768857142857142857142857142834533'),
   mpf('0.00998537240146541587344180305938211'),
   mpf('0.00976309620789022465053059414510641')],
  0.11: [mpf('0.709571428571428571428571428543019'),
   mpf('0.0107194963658037674442929138527877'),
   mpf('0.0105462334148894203116648376529877')],
  0.12: [mpf('0.650142857142857142857142857108635'),
   mpf('0.011232697419106383815038238251